### A08: Bootstrapping

María Fernanda Muñoz Sevilla
751190


In [49]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real

## Avertising

In [3]:
ad= pd.read_csv("C:/Users/munoz/Downloads/lab_apre_est/Advertising.csv")
ad.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [4]:
np.random.seed(0)

In [32]:
x1 = ad.TV.values.reshape([-1,1])
x2 = ad.radio.values.reshape([-1,1])
x3 = ad.newspaper.values.reshape([-1,1])
y1= ad.sales
n1=len(y1)
unos=np.ones([n1,1])
X=np.hstack([unos,x1,x2,x3])
ols=sm.OLS(y1,X)
resultados=ols.fit()
resultados.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           1.58e-96
Time:                        00:11:45   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
n_boot=1000
coef_sample=np.zeros((n_boot,X.shape[1]))
for i in range(n_boot):
    sample_indices=np.random.choice(range(len(y1)), size=len(y1), replace=True)
    X_sample=X[sample_indices]
    y_sample=y1.iloc[sample_indices]
    ols=sm.OLS(y_sample, X_sample)
    resultados=ols.fit()
    coef_sample[i,:]=resultados.params
coef_means=np.mean(coef_sample, axis=0)
print("Coeficientes medios: ", coef_means)
#desviación
coef_stds=np.std(coef_sample, axis=0)
print("Desviación estándar: ", coef_stds)

Coeficientes medios:  [ 2.93840154e+00  4.56893553e-02  1.88966780e-01 -8.65812985e-04]
Desviación estándar:  [0.3457395  0.00193279 0.01075342 0.0062605 ]


Los resultados del bootstrap muestran que el modelo es bastante estable y que los coeficientes estimados no varían demasiado entre las distintas muestras. El coeficiente medio de la intercepto es aproximadamente 2.95, con una desviación estándar de 0.35, lo que indica que el punto de partida del modelo es consistente. El segundo coeficiente tiene un valor medio de 0.045 y una desviación estándar muy pequeña (0.0019), lo que sugiere que este predictor tiene un efecto positivo y muy estable. El tercer coeficiente también es positivo, con un valor promedio de 0.188 y una desviación estándar baja (0.0107), lo que indica que su influencia en el resultado es confiable. En cambio, el cuarto coeficiente es ligeramente negativo (–0.0012) y presenta una desviación estándar pequeña (0.0065), lo que implica que su efecto es muy débil y probablemente no significativo. En conjunto, las pequeñas desviaciones estándar reflejan que los coeficientes del modelo son robustos y que los efectos estimados se mantienen consistentes incluso al re-muestrear los datos múltiples veces.

## Default

In [14]:
defa=pd.read_csv("C:/Users/munoz/Downloads/lab_apre_est/Default.csv")
defa.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880


In [16]:
stu=(defa["student"]=="Yes").astype(int)
x1=defa["balance"].values.reshape(-1,1)
x3=stu.values.reshape(-1,1)
x2=defa["income"].values.reshape(-1,1)
y=defa["default"]== "Yes"
# Matriz de predictores
X3=np.hstack([x1,x2,x3])

modelo=LogisticRegression()
modelo.fit(X3,y)

#Intercepto y coeficientes
beta0_3=modelo.intercept_
beta1_3=modelo.coef_

print("Intercept: ",beta0_3)
print("Coeficientes: ",beta1_3)


Intercept:  [-10.90178652]
Coeficientes:  [[ 5.73059385e-03  3.96180616e-06 -6.12575695e-01]]


In [18]:
defa['student_Yes'] = pd.get_dummies(defa['student'], drop_first=True)
X3 = defa[['balance', 'income', 'student_Yes']]
y= defa['default']=='Yes'
X_multi = LogisticRegression()
X_multi.fit(X3, y)


LogisticRegression()

In [26]:
# Convertir X3 a float
X3 = X3.astype(float)
y = (defa['default'] == 'Yes').astype(int)

n_boot = 1000
coef_samples = np.zeros((n_boot, X3.shape[1] + 1))  # +1 para intercepto

for i in range(n_boot):
    sample_indices = np.random.choice(range(len(y)), size=len(y), replace=True)
    X_sample = X3.iloc[sample_indices]
    y_sample = y.iloc[sample_indices]

    # statsmodels requiere float explícito
    X_sample_const = sm.add_constant(X_sample).astype(float)

    logit = sm.Logit(y_sample, X_sample_const)
    resultados = logit.fit(disp=0)
    coef_samples[i, :] = resultados.params

coef_means = np.mean(coef_samples, axis=0)
coef_stds = np.std(coef_samples, axis=0)

print("Coeficientes medios:", coef_means)
print("Desviación estándar:", coef_stds)


Coeficientes medios: [-1.09030668e+01  5.75806186e-03  2.94621116e-06 -6.51737861e-01]
Desviación estándar: [4.85443570e-01 2.32164632e-04 8.35960030e-06 2.40458595e-01]


Los resultados del bootstrap muestran que el modelo es relativamente estable y que los coeficientes estimados no varían demasiado entre los distintos remuestreos. El intercepto tiene un valor promedio de aproximadamente –10.90, con una desviación estándar de 0.485, lo que indica que el punto de partida del modelo (log-odds cuando todas las variables predictoras son cero) es consistente. El primer predictor tiene un coeficiente medio de 0.00576 con una desviación estándar muy pequeña (0.00023), lo que sugiere que su efecto positivo sobre la variable de respuesta es pequeño pero muy estable. El segundo predictor tiene un valor promedio casi nulo (2.95e-06) con una desviación estándar igualmente mínima (8.36e-06), indicando que su influencia sobre el resultado es prácticamente despreciable. Finalmente, el cuarto coeficiente es negativo (–0.6517) y su desviación estándar es 0.240, lo que muestra que reduce el log-odds de forma moderada y que su efecto es relativamente estable, aunque algo más variable que los coeficientes anteriores. En conjunto, estas desviaciones estándar bajas reflejan que los coeficientes del modelo son robustos, mientras que los valores cercanos a cero sugieren que algunas variables tienen poca o ninguna influencia sobre la probabilidad de que ocurra el evento modelado.

## Avertising con penalización de Ridge (L2)

In [40]:
alpha = 1.0
coef_ridge = np.zeros((n_boot, X.shape[1]))

for i in range(n_boot):
    sample_indices = np.random.choice(range(len(y1)), size=len(y1), replace=True)
    X_sample = X[sample_indices]
    y_sample = y1.iloc[sample_indices]

    ridge = Ridge(alpha=alpha)
    ridge.fit(X_sample, y_sample)

    # Guardas intercepto y coeficientes si existen
    coef_ridge[i, 0] = ridge.intercept_
    coef_ridge[i, :] = ridge.coef_
    
coef_means_ridge = np.mean(coef_ridge, axis=0)
print("Coeficientes medios Ridge:", coef_means_ridge)

coef_stds_ridge = np.std(coef_ridge, axis=0)
print("Desviación estándar Ridge:", coef_stds_ridge)


Coeficientes medios Ridge: [ 0.          0.04566245  0.18840823 -0.00038391]
Desviación estándar Ridge: [0.         0.001854   0.01102101 0.00635759]


Los resultados del bootstrap aplicando Ridge muestran que la penalización reduce la magnitud de los coeficientes y estabiliza el modelo. El intercepto es cero debido a la penalización aplicada al modelo, lo que indica que el ajuste inicial se centra en minimizar la varianza de los coeficientes. El coeficiente de la variable TV es 0.0457 con una desviación estándar muy baja (0.00185), lo que indica que su efecto positivo sobre las ventas es consistente y estable a través de los remuestreos. El coeficiente de Radio es 0.1884 con desviación estándar 0.0110, mostrando también un efecto positivo claro y relativamente estable. El coeficiente de Newspaper es prácticamente cero (–0.00038) con desviación estándar 0.00636, lo que indica que su influencia sobre las ventas es muy pequeña y prácticamente despreciable. En general, el uso de Ridge reduce la variabilidad de los coeficientes y confirma que las variables TV y Radio son los predictores más relevantes para las ventas, mientras que Newspaper no aporta información significativa. La baja desviación estándar de todos los coeficientes refleja que el modelo es robusto frente a la variabilidad de la muestra.

In [53]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

search_spaces = {
    'alpha': Real(1e-2, 100, prior='log-uniform')
}
bayes_search = BayesSearchCV(
    estimator=Ridge(),
    search_spaces=search_spaces,
    n_iter=50,              
    cv=10,                   
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42
)

bayes_search.fit(X_scaled, y1)

best_model = bayes_search.best_estimator_
best_alpha = bayes_search.best_params_['alpha']

coefs_base = [best_model.intercept_] + list(best_model.coef_.ravel())
medias_ridge = np.mean(coefs_base, axis=0)
std_ridge = np.std(coefs_base, axis=0)

print("Mejor alpha :",best_alpha)
print("La media de los coeficientes con ridge es de: ",medias_ridge)
print("La desviacion estandar de los coeficientes con ridge es de: ",std_ridge)



Mejor alpha : 1.1692964055498163
La media de los coeficientes con ridge es de:  4.13578989455288
La desviacion estandar de los coeficientes con ridge es de:  5.176692343933836


La optimización bayesiana sobre el modelo Ridge arrojó un mejor valor de alpha de aproximadamente 1.17, lo que indica que se aplicó una penalización moderada a los coeficientes para reducir su magnitud y estabilizar el modelo. Al analizar los coeficientes obtenidos del modelo con este alpha, se observa que la media de los coeficientes es de 4.14 y la desviación estándar es de 5.18. Sin embargo, estos valores reflejan únicamente los coeficientes de un único modelo ajustado y no su variabilidad real frente a distintas muestras, por lo que no representan la incertidumbre del modelo de manera robusta. Aun así, el hecho de que los coeficientes sean relativamente pequeños y la penalización aplicada sugiere que Ridge ha reducido la magnitud de los predictores, ayudando a evitar sobreajuste y a estabilizar la estimación de los efectos de las variables sobre la variable de respuesta. En términos generales, la penalización logra que los coeficientes sean más conservadores y el modelo más robusto frente a la variabilidad de los datos.

In [64]:
n_iterations = 1000
coefs_ridge_bootstrap = []
model_ridge = Ridge(alpha=best_alpha)


for i in range(n_iterations):
    X_res, y_res = resample(X_scaled, y1.values, replace=True, random_state=i)

    model_ridge.fit(X_res, y_res)

    iter_coefs = [model_ridge.intercept_] + list(model_ridge.coef_.ravel())
    coefs_ridge_bootstrap.append(iter_coefs)

coefs_ridge_bootstrap = np.array(coefs_ridge_bootstrap)
medias_bootstrap = np.mean(coefs_ridge_bootstrap, axis=0)
std_bootstrap = np.std(coefs_ridge_bootstrap, axis=0)

print("La media de los coeficientes con ridge y bootstarp es de: ", medias_bootstrap)
print("La desviacion estandar de los coeficientes con ridge y bootstarp es de: ",std_bootstrap)

La media de los coeficientes con ridge y bootstarp es de:  [ 1.40272104e+01  0.00000000e+00  3.88770035e+00  2.76841086e+00
 -4.08525015e-03]
La desviacion estandar de los coeficientes con ridge y bootstarp es de:  [0.11987603 0.         0.1625338  0.16124048 0.1362477 ]


Los resultados del bootstrap aplicado al modelo Ridge muestran que la penalización ha reducido la magnitud y la variabilidad de los coeficientes, haciendo que el modelo sea más estable y menos propenso a sobreajuste. El intercepto tiene un valor medio de aproximadamente 14.03 con una desviación estándar baja (0.12), lo que indica que el punto de partida del modelo es consistente a través de los remuestreos. El segundo coeficiente es exactamente cero con desviación estándar cero, lo que refleja que la variable correspondiente fue completamente eliminada o reducida a cero por la penalización, y por lo tanto no aporta información significativa al modelo. Los coeficientes de las siguientes variables son positivos (3.89 y 2.77) y presentan desviaciones estándar moderadamente bajas (0.16 y 0.16), lo que indica que tienen un efecto estable y relevante sobre la variable de respuesta. Finalmente, el último coeficiente es cercano a cero (–0.004) con desviación estándar 0.14, sugiriendo que su efecto es prácticamente despreciable y relativamente inestable en comparación con las variables más influyentes. En conjunto, estos resultados muestran que Ridge ha permitido identificar claramente cuáles predictores tienen un impacto relevante y consistente, mientras que las variables menos importantes fueron penalizadas, estabilizando así el modelo frente a la variabilidad de la muestra.